# Лабораторная работа: Мета-обучение
## Шаг 2: Извлечение мета-признаков и эксперимент с перестановками

**Цель:** Реализовать мета-признаки и проверить их инвариантность к перестановкам

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris, load_wine, load_breast_cancer
from sklearn.utils import shuffle

from src.meta_features.extractor import MetaFeatureExtractor
from src.meta_features.general import GeneralFeatures
from src.meta_features.statistical import StatisticalFeatures
from src.meta_features.structural import StructuralFeatures
from src.meta_features.landmarking import LandmarkingFeatures
from src.experiments.permutation import PermutationExperiment

%matplotlib inline
sns.set_style('whitegrid')

In [ ]:
# Загружаем список датасетов
datasets_df = pd.read_csv('../data/raw/dataset_list.csv')
print(f"Загружено {len(datasets_df)} датасетов")

### 2.1 Тестирование извлечения мета-признаков на примере Iris

In [ ]:
# Загружаем Iris
iris = load_iris()
X_iris, y_iris = iris.data, iris.target

print(f"Iris dataset: {X_iris.shape[0]} объектов, {X_iris.shape[1]} признаков, {len(np.unique(y_iris))} классов")

In [ ]:
# Извлекаем мета-признаки
extractor = MetaFeatureExtractor()
iris_features = extractor.extract(X_iris, y_iris, dataset_id="iris")

print("\nИзвлеченные мета-признаки для Iris:")
for key, value in list(iris_features.items())[:15]:  # покажем первые 15
    print(f"  {key}: {value:.4f}" if isinstance(value, float) else f"  {key}: {value}")

### 2.2 Эксперимент с перестановками

In [ ]:
# Создаем эксперимент
perm_exp = PermutationExperiment(extractor)

# Запускаем на Iris
perm_results = perm_exp.run_experiment(X_iris, y_iris)

# Анализируем изменения
changes_df = perm_exp.analyze_changes(perm_results)
changes_df.head(10)

In [ ]:
# Визуализируем изменения
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Boxplot изменений по типам перестановок
sns.boxplot(data=changes_df, x='experiment', y='relative_change', ax=axes[0])
axes[0].set_xlabel('Тип перестановки')
axes[0].set_ylabel('Относительное изменение')
axes[0].set_title('Изменения мета-признаков при перестановках')
axes[0].tick_params(axis='x', rotation=45)

# Средние изменения по типам
mean_changes = changes_df.groupby('experiment')['relative_change'].mean()
mean_changes.plot(kind='bar', ax=axes[1], color='coral', alpha=0.7)
axes[1].set_xlabel('Тип перестановки')
axes[1].set_ylabel('Среднее относительное изменение')
axes[1].set_title('Средние изменения по типам перестановок')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Анализ по группам признаков
changes_df['feature_group'] = changes_df['feature'].apply(
    lambda x: 'статистические' if any(t in x for t in ['mean', 'sd', 'corr'])
    else ('структурные' if any(t in x for t in ['entropy', 'mutual'])
          else 'базовые')
)

group_changes = changes_df.groupby(['experiment', 'feature_group'])['relative_change'].mean().unstack()
group_changes

In [ ]:
# Визуализация по группам
group_changes.plot(kind='bar', figsize=(12, 6))
plt.xlabel('Тип перестановки')
plt.ylabel('Среднее относительное изменение')
plt.title('Изменения мета-признаков по группам')
plt.legend(title='Группа признаков')
plt.tight_layout()
plt.show()

### 2.3 Массовое извлечение мета-признаков

In [ ]:
# Загружаем первые 5 датасетов для теста (в реальности будет 350)
test_datasets = datasets_df.head(5)

meta_features_list = []

for idx, row in test_datasets.iterrows():
    dataset_id = row['did']
    print(f"Обработка датасета {dataset_id}: {row['name']}")
    
    # Загружаем данные
    from src.data.collector import OpenMLCollector
    collector = OpenMLCollector(use_cache=True)
    X, y = collector.download_dataset(dataset_id)
    
    if X is not None:
        # Извлекаем признаки
        features = extractor.extract(X, y, dataset_id=str(dataset_id))
        features['dataset_id'] = dataset_id
        features['dataset_name'] = row['name']
        meta_features_list.append(features)
        print(f"  Извлечено {len(features)} признаков")
    else:
        print(f"  Ошибка загрузки")

In [ ]:
# Создаем DataFrame
meta_df = pd.DataFrame(meta_features_list)
print(f"\nСоздан DataFrame размером {meta_df.shape}")
meta_df.head()

### 2.4 Выводы по шагу 2

In [ ]:
print("="*60)
print("ВЫВОДЫ ПО ШАГУ 2: МЕТА-ПРИЗНАКИ")
print("="*60)
print("""
1. РЕАЛИЗОВАНЫ ГРУППЫ МЕТА-ПРИЗНАКОВ:
   - Базовые: количество объектов, признаков, классов, размерность
   - Статистические: среднее, стандартное отклонение, асимметрия, корреляции
   - Структурные: энтропия классов, взаимная информация
   - Ландмаркинг: производительность простых моделей

2. ЭКСПЕРИМЕНТ С ПЕРЕСТАНОВКАМИ:
   - Перемешивание строк: минимальные изменения (мета-признаки инвариантны)
   - Перемешивание столбцов: значительные изменения
   - Перестановка меток: изменения в структурных признаках

3. МАСШТАБИРУЕМОСТЬ:
   - Механизм кэширования для ускорения повторных вычислений
   - Возможность параллельной обработки
""")